
# Lukumääriä ja prosentteja

Parhaat käytännöt lukumäärä- ja prosentti-yhteenvetojen laskemiseen eri tilanteissa.

In [1]:
import pandas as pd

In [2]:
#Avaan datan dataframeen.
#Vanhemmissa pandas-versioissa pitää käyttää sheet_name sijasta sheetname.
df = pd.read_excel('http://taanila.fi/data1.xlsx', sheet_name = 'Data')

#Tulostan viisi ensimmäistä riviä:
df.head()

,nro,sukup,ikä,perhe,koulutus,palveluv,palkka,johto,työtov,työymp,palkkat,työteht,työterv,lomaosa,kuntosa,hieroja
0,1,1,38,1,1.0,22.0,3587,3,3.0,3,3,3,NaN,NaN,NaN,NaN
1,2,1,29,2,2.0,10.0,2963,1,5.0,2,1,3,NaN,NaN,NaN,NaN
2,3,1,30,1,1.0,7.0,1989,3,4.0,1,1,3,1.0,NaN,NaN,NaN
3,4,1,36,2,1.0,14.0,2144,3,3.0,3,3,3,1.0,NaN,NaN,NaN
4,5,1,24,1,2.0,4.0,2183,2,3.0,2,1,2,1.0,NaN,NaN,NaN


In [3]:
#Muutamia listoja jatkoa varten:
koulutus = ['Peruskoulu','2.aste','Korkeakoulu','Ylempi korkeakoulu']
mielipideasteikko = ['Erittäin tyytymätön','Tyytymätön','Neutraali',
          'Tyytyväinen','Erittäin tyytyväinen']
tyytyväisyydet = ['Johtoon','Työtovereihin','Työympäristöön','Palkkaan','Työtehtäviin']
monivalinnat = ['Kuntosali','Lomaosake','Hieroja','Työterveyshuolto']

In [4]:
#Frekvenssit value_counts-funktiolla:
df1 = df['koulutus'].value_counts(sort = False).to_frame()

#Otsikot kuntoon
df1.index = koulutus

df1

,koulutus
Peruskoulu,27
2.aste,30
Korkeakoulu,22
Ylempi korkeakoulu,2


In [5]:
#Prosentit frekvenssien viereen:
df1 = df1.rename(columns = {'koulutus': 'Lkm'})
df1['%'] = df1['Lkm'] / df1['Lkm'].sum() * 100

df1

,Lkm,%
Peruskoulu,27,33.333333
2.aste,30,37.037037
Korkeakoulu,22,27.160494
Ylempi korkeakoulu,2,2.469136


In [6]:
#Kertymäprosentit ja pyöristys yhteen desimaaliin:
df1['Kertymä-%'] = df1['%'].cumsum().round(decimals = 1)
df1['%'] = df1['%'].round(decimals = 1)

df1

,Lkm,%,Kertymä-%
Peruskoulu,27,33.3,33.3
2.aste,30,37.0,70.4
Korkeakoulu,22,27.2,97.5
Ylempi korkeakoulu,2,2.5,100.0


In [7]:
#crosstab-funktio on vaihtoehtoinen tapa frekvenssien laskemiseen.
#crosstabin etu on, että se tuottaa suoraan dataframen.
df2 = pd.crosstab(df['koulutus'], 'Lkm')

#Otsikot kuntoon:
df2.index = koulutus
df2.columns.name = '' #poistaa häiritsevän col_0 -otsikon taulukon oikeasta yläkulmasta

df2

,Lkm
Peruskoulu,27
2.aste,30
Korkeakoulu,22
Ylempi korkeakoulu,2


In [8]:
#Prosentit frekvenssien viereen:
df2['%'] = df2['Lkm'] / df2['Lkm'].sum()*100

df2

,Lkm,%
Peruskoulu,27,33.333333
2.aste,30,37.037037
Korkeakoulu,22,27.160494
Ylempi korkeakoulu,2,2.469136


In [9]:
#Kertymäprosentit:
df2['Kertymä-%'] = df2['%'].cumsum()

#Loppusilaus:
df2.style.format({'%': '{:.1f} %', 'Kertymä-%': '{:.1f} %'})

,Lkm,%,Kertymä-%
Peruskoulu,27,33.3 %,33.3 %
2.aste,30,37.0 %,70.4 %
Korkeakoulu,22,27.2 %,97.5 %
Ylempi korkeakoulu,2,2.5 %,100.0 %


In [10]:
#Ikäluokkien frekvenssit:
df3 = df['ikä'].value_counts(sort = False, bins = [18,29,39,49,59,69]).to_frame()

#n-arvo (tarvitsen seuraavassa solussa)
n = df3['ikä'].sum()

df3

,ikä
"(17.999, 29.0]",17
"(29.0, 39.0]",30
"(39.0, 49.0]",23
"(49.0, 59.0]",11
"(59.0, 69.0]",1


In [11]:
#Ikäluokkien frekvenssiprosentit ja kertymäprosentit:
df4 = df['ikä'].value_counts(normalize = True, sort = False, bins = [18,29,39,49,59,69]).to_frame() * 100

#Kertymäprosentit:
df4['Kertymä-%'] = df4['ikä'].cumsum()

#Otsikon vaihdos:
df4 = df4.rename(columns = {'ikä':'%, n='+ str(n)})
#df4.columns = ['%' + str(n), 'Kertymä-%']

#Loppusilaus
df4.style.format('{:.1f} %')

,"%, n=82",Kertymä-%
"(17.999, 29.0]",20.7 %,20.7 %
"(29.0, 39.0]",36.6 %,57.3 %
"(39.0, 49.0]",28.0 %,85.4 %
"(49.0, 59.0]",13.4 %,98.8 %
"(59.0, 69.0]",1.2 %,100.0 %


In [12]:
#Monivalintakysymyksen valintojen lukumäärät suuruusjärjestyksessä:
df5 = df[['työterv','lomaosa','kuntosa','hieroja']].sum().sort_values(ascending = False).to_frame()

#Otsikot kuntoon:
df5 = df5.rename(columns = {0: 'Lkm'})
df5.index=monivalinnat

#Loppusilaus:
df5.style.format('{:.0f}')

,Lkm
Kuntosali,47
Lomaosake,22
Hieroja,20
Työterveyshuolto,9


In [13]:
#Prosenttia vastaajien kokonaismäärästä:
df5['% vastaajista'] = df5['Lkm'] / df.shape[0] *100

#Loppusilaus:
df5.style.format({'% vastaajista':'{:.1f} %'})

,Lkm,% vastaajista
Kuntosali,47,57.3 %
Lomaosake,22,26.8 %
Hieroja,20,24.4 %
Työterveyshuolto,9,11.0 %


In [14]:
#Ristiintaulukointi:
df6 = pd.crosstab(df['koulutus'], df['sukup'])

#Otsikot kuntoon:
df6.index = koulutus
df6.columns = ['Mies','Nainen']

#Miesten ja naisten lukumäärät (tarvitsen seuraavassa solussa):
miehet = df6['Mies'].sum()
naiset = df6['Nainen'].sum()

df6

,Mies,Nainen
Peruskoulu,22,5
2.aste,23,7
Korkeakoulu,15,7
Ylempi korkeakoulu,2,0


In [15]:
#Ristiintaulukointi prosentteina:
df7 = pd.crosstab(df['koulutus'], df['sukup'], margins = True, normalize = 'columns') * 100

#Otsikot kuntoon:
df7.index = koulutus

#Kikkaillaan n-arvot mukaan:
df7.columns = ['Mies, n=' + str(miehet),'Nainen, n=' + str(naiset),
               'Yhteensä, n=' + str(miehet+naiset)]

#Loppusilaus:
df7.style.format('{:.1f} %')

,"Mies, n=62","Nainen, n=19","Yhteensä, n=81"
Peruskoulu,35.5 %,26.3 %,33.3 %
2.aste,37.1 %,36.8 %,37.0 %
Korkeakoulu,24.2 %,36.8 %,27.2 %
Ylempi korkeakoulu,3.2 %,0.0 %,2.5 %


In [16]:
#Ikäluokan ja sukupuolen ristiintaulukointi:
df['ikäluokka'] = pd.cut(df['ikä'], bins = [18,29,39,49,59,69])
df8 = pd.crosstab(df['ikäluokka'], df['sukup'])

#Otsikot kuntoon:
df8.columns = ['Mies','Nainen']
df8.index.name='Ikäluokka'

df8

,Mies,Nainen
Ikäluokka,,
"(18, 29]",16,1
"(29, 39]",23,7
"(39, 49]",16,7
"(49, 59]",7,4
"(59, 69]",1,0


In [17]:
#Monivalinnan vaihtoehtojen lukumäärät sukupuolen mukaan:
df9 = df.groupby('sukup')['työterv','lomaosa','kuntosa','hieroja'].sum()

#Otsikot kuntoon:
df9.index = ['Mies','Nainen']
df9.columns = monivalinnat

#Loppusilaus:
df9.style.format('{:.0f}')

,Kuntosali,Lomaosake,Hieroja,Työterveyshuolto
Mies,35,16,8,13
Nainen,12,4,1,9


In [18]:
#Prosentit sukupuolesta:
df10 = df.groupby('sukup')['työterv','lomaosa','kuntosa','hieroja'].sum() * 100

#Otsikot kuntoon:
df10.index = ['Mies','Nainen']
df10.columns = monivalinnat

#Miesten ja naisten lukumäärät:
miehet = df['sukup'].value_counts()[1]
naiset = df['sukup'].value_counts()[2]

#Prosentit:
df10.iloc[0] = df10.iloc[0] / miehet
df10.iloc[1] = df10.iloc[1] / naiset

#Kikkaillaan n-arvot mukaan:
df10.index = ['Mies, n=' + str(miehet) ,'Nainen, n=' + str(naiset)]

#Loppusilaus:
df10.style.format('{:.1f} %')

,Kuntosali,Lomaosake,Hieroja,Työterveyshuolto
"Mies, n=63",55.6 %,25.4 %,12.7 %,20.6 %
"Nainen, n=19",63.2 %,21.1 %,5.3 %,47.4 %
